In [ ]:
import h5py
import pickle as pkl
import numpy as np
import os

In [ ]:
dataset_dir = 'D:/Users/mickey/Data/datasets/ecog/goose_wireless'
assert os.path.exists(dataset_dir), f'{dataset_dir} not found!'

In [ ]:
full_dataset_h5f = h5py.File(os.path.join(dataset_dir,'gw_250'),mode='r')
with open(os.path.join(dataset_dir,'gw_250_param.pkl'),mode='rb') as pf:
    full_dataset_param_dict = pkl.load(pf)

In [ ]:
# create smaller dataset, randomly sampling from each portion
frac_train_size = full_dataset_h5f['train_ecog'].shape[0]//10
frac_valid_size = full_dataset_h5f['valid_ecog'].shape[0]//10
frac_test_size = full_dataset_h5f['test_ecog'].shape[0]//10
frac_train_idx = np.random.permutation(full_dataset_h5f['train_ecog'].shape[0])[:frac_train_size]
frac_valid_idx = np.random.permutation(full_dataset_h5f['valid_ecog'].shape[0])[:frac_valid_size]
frac_test_idx = np.random.permutation(full_dataset_h5f['test_ecog'].shape[0])[:frac_test_size]

In [ ]:
# save dataset and parameter pickle (same as the full set, I didn't actually save the sample order in the original).
frac_dataset_h5f = h5py.File(os.path.join(dataset_dir,'gw_250_pilot'),mode='w')
frac_dataset_h5f['train_ecog'] = full_dataset_h5f['train_ecog'].value[frac_train_idx,:,:]
frac_dataset_h5f['valid_ecog'] = full_dataset_h5f['valid_ecog'].value[frac_valid_idx,:,:]
frac_dataset_h5f['test_ecog'] = full_dataset_h5f['test_ecog'].value[frac_test_idx]

In [ ]:
full_dataset_h5f.close()
frac_dataset_h5f.close()

In [ ]:
with open(os.path.join(dataset_dir,'gw_250_pilot_param.pkl'),mode='wb') as pf:
    pkl.dump(full_dataset_param_dict,pf)